<a href="https://colab.research.google.com/github/subod4/NLP/blob/main/Text_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
text = "Pollution is a pressing issue affecting our planet today, encompassing various forms of contaminants that degrade the environment and pose risks to human health. From air and water pollution to soil and noise pollution, its effects are far-reaching and demand immediate attention. Major contributors to pollution include industrial activities, vehicular emissions, agricultural practices, and improper waste disposal, which release harmful substances like carbon dioxide, sulfur dioxide, nitrogen oxides, and pesticides into the environment, contaminating air, water, and soil. Additionally, deforestation and urbanization exacerbate the problem, disrupting ecosystems and natural balance. Pollution has detrimental effects on both the environment and human health, leading to global warming, acid rain, depletion of the ozone layer, and loss of biodiversity. Polluted air and water cause respiratory diseases, cancer, and other health complications, while contaminated soil affects agricultural productivity, further jeopardizing food security. In conclusion, pollution poses a significant threat to the well-being of our planet and its inhabitants. Addressing this issue requires collective efforts from governments, industries, and individuals. Implementing sustainable practices, investing in clean technologies, and raising awareness are crucial steps towards mitigating pollution and preserving the environment for future generations."

In [2]:
len(text)

1440

In [3]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [4]:
nlp = spacy.load('en_core_web_sm')

In [6]:
doc = nlp(text)

In [9]:
tokens = [token.text.lower() for token in doc
          if not token.is_stop and
          not token.is_punct and token.text != '/n']


In [10]:
tokens

['pollution',
 'pressing',
 'issue',
 'affecting',
 'planet',
 'today',
 'encompassing',
 'forms',
 'contaminants',
 'degrade',
 'environment',
 'pose',
 'risks',
 'human',
 'health',
 'air',
 'water',
 'pollution',
 'soil',
 'noise',
 'pollution',
 'effects',
 'far',
 'reaching',
 'demand',
 'immediate',
 'attention',
 'major',
 'contributors',
 'pollution',
 'include',
 'industrial',
 'activities',
 'vehicular',
 'emissions',
 'agricultural',
 'practices',
 'improper',
 'waste',
 'disposal',
 'release',
 'harmful',
 'substances',
 'like',
 'carbon',
 'dioxide',
 'sulfur',
 'dioxide',
 'nitrogen',
 'oxides',
 'pesticides',
 'environment',
 'contaminating',
 'air',
 'water',
 'soil',
 'additionally',
 'deforestation',
 'urbanization',
 'exacerbate',
 'problem',
 'disrupting',
 'ecosystems',
 'natural',
 'balance',
 'pollution',
 'detrimental',
 'effects',
 'environment',
 'human',
 'health',
 'leading',
 'global',
 'warming',
 'acid',
 'rain',
 'depletion',
 'ozone',
 'layer',
 'loss',

In [11]:
tokens1 = []
stopwords = list(STOP_WORDS)
allowed_pos = ['ADJ', 'PROPN', 'VERB', 'NOUN']
for token in doc:
    if token.text not in stopwords and token.text not in punctuation:
        if token.pos_ in allowed_pos:
            tokens1.append(token.text)

In [12]:
tokens1

['Pollution',
 'pressing',
 'issue',
 'affecting',
 'planet',
 'today',
 'encompassing',
 'forms',
 'contaminants',
 'degrade',
 'environment',
 'pose',
 'risks',
 'human',
 'health',
 'air',
 'water',
 'pollution',
 'soil',
 'noise',
 'pollution',
 'effects',
 'reaching',
 'demand',
 'immediate',
 'attention',
 'Major',
 'contributors',
 'pollution',
 'include',
 'industrial',
 'activities',
 'vehicular',
 'emissions',
 'agricultural',
 'practices',
 'improper',
 'waste',
 'disposal',
 'release',
 'harmful',
 'substances',
 'carbon',
 'dioxide',
 'sulfur',
 'dioxide',
 'nitrogen',
 'oxides',
 'pesticides',
 'environment',
 'contaminating',
 'air',
 'water',
 'soil',
 'deforestation',
 'urbanization',
 'exacerbate',
 'problem',
 'disrupting',
 'ecosystems',
 'natural',
 'balance',
 'Pollution',
 'detrimental',
 'effects',
 'environment',
 'human',
 'health',
 'leading',
 'global',
 'warming',
 'acid',
 'rain',
 'depletion',
 'ozone',
 'layer',
 'loss',
 'biodiversity',
 'Polluted',
 'a